In [1]:
import json, math, copy
from geosnap.data import store_ltdb
from geosnap.data import Community
import pandas as pd
import shapely.wkt
import shapely.geometry
from datetime import datetime
from datetime import timedelta
from pathlib import Path
from geosnap.data import store_census
from geosnap.data import data_store
from geosnap.data import data_store as store
from pathlib import Path
from geosnap.data import store_ltdb

In [2]:
SanDiego = Community.from_ltdb(msa_fips='41740')
SanDiego.gdf

,geoid,n_asian_under_15,n_black_under_15,n_hispanic_under_15,n_native_under_15,n_white_under_15,n_persons_under_18,n_asian_over_60,n_black_over_60,n_hispanic_over_60,...,year,n_total_housing_units_sample,p_nonhisp_white_persons,p_white_over_60,p_black_over_60,p_hispanic_over_60,p_native_over_60,p_asian_over_60,p_disabled,geometry
0,06073000100,NaN,2.061758,NaN,NaN,874.357666,1036.023204,NaN,0.000000,NaN,...,1970,1248.033948,NaN,22.852687,0.000000,NaN,NaN,NaN,7.212085,"POLYGON ((-117.194848 32.754465, -117.191848 3..."
1,06073000201,NaN,0.863271,NaN,NaN,328.220536,372.341146,NaN,0.286230,NaN,...,1970,779.659146,NaN,20.442235,0.016707,NaN,NaN,NaN,6.454919,"POLYGON ((-117.178867 32.75765, -117.177058 32..."
2,06073000202,NaN,2.152735,NaN,NaN,818.482141,928.505557,NaN,0.713770,NaN,...,1970,1944.232739,NaN,20.442235,0.016707,NaN,NaN,NaN,6.454919,"POLYGON ((-117.183505 32.746641, -117.181316 3..."
3,06073000300,NaN,6.000000,NaN,NaN,386.000000,453.000000,NaN,1.000000,NaN,...,1970,2145.000000,NaN,32.348957,0.027093,NaN,NaN,NaN,8.507180,"POLYGON ((-117.168645 32.748968, -117.166015 3..."
4,06073000400,NaN,7.790199,NaN,NaN,259.997894,326.214600,NaN,0.973775,NaN,...,1970,1679.761719,NaN,24.083417,0.033636,NaN,NaN,NaN,8.442650,"POLYGON ((-117.170867 32.75865, -117.168156 32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,06073021800,0.0,0.000000,108.0,0.0,476.000000,682.000000,NaN,NaN,NaN,...,2010,1137.000000,91.686367,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.192907 32.6879, -117.191676 32...."
3132,06073021900,76.0,18.000000,258.0,0.0,39.000000,498.000000,NaN,NaN,NaN,...,2010,681.000000,30.622204,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.135422 32.678614, -117.125674 3..."
3133,06073022000,189.0,0.000000,764.0,0.0,16.000000,1209.000000,NaN,NaN,NaN,...,2010,1248.000000,7.371112,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.093518 32.68721, -117.090508 32..."
3134,06073022100,156.0,19.000000,788.0,0.0,1234.000000,2714.000000,NaN,NaN,NaN,...,2010,3738.000000,67.584301,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.33373 33.144331, -117.31618 33...."


In [3]:
cluster = SanDiego.cluster(
    columns=['p_asian_persons'],
    n_clusters = 10,
    method ='kmeans',
)
cluster.gdf

,year,geoid,n_asian_under_15,n_black_under_15,n_hispanic_under_15,n_native_under_15,n_white_under_15,n_persons_under_18,n_asian_over_60,n_black_over_60,...,n_total_housing_units_sample,p_nonhisp_white_persons,p_white_over_60,p_black_over_60,p_hispanic_over_60,p_native_over_60,p_asian_over_60,p_disabled,geometry,kmeans
0,1970,06073000100,NaN,2.061758,NaN,NaN,874.357666,1036.023204,NaN,0.000000,...,1248.033948,NaN,22.852687,0.000000,NaN,NaN,NaN,7.212085,"POLYGON ((-117.194848 32.754465, -117.191848 3...",9
1,1970,06073000201,NaN,0.863271,NaN,NaN,328.220536,372.341146,NaN,0.286230,...,779.659146,NaN,20.442235,0.016707,NaN,NaN,NaN,6.454919,"POLYGON ((-117.178867 32.75765, -117.177058 32...",9
2,1970,06073000202,NaN,2.152735,NaN,NaN,818.482141,928.505557,NaN,0.713770,...,1944.232739,NaN,20.442235,0.016707,NaN,NaN,NaN,6.454919,"POLYGON ((-117.183505 32.746641, -117.181316 3...",9
3,1970,06073000300,NaN,6.000000,NaN,NaN,386.000000,453.000000,NaN,1.000000,...,2145.000000,NaN,32.348957,0.027093,NaN,NaN,NaN,8.507180,"POLYGON ((-117.168645 32.748968, -117.166015 3...",0
4,1970,06073000400,NaN,7.790199,NaN,NaN,259.997894,326.214600,NaN,0.973775,...,1679.761719,NaN,24.083417,0.033636,NaN,NaN,NaN,8.442650,"POLYGON ((-117.170867 32.75865, -117.168156 32...",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,2010,06073021800,0.0,0.000000,108.0,0.0,476.000000,682.000000,NaN,NaN,...,1137.000000,91.686367,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.192907 32.6879, -117.191676 32....",9
3132,2010,06073021900,76.0,18.000000,258.0,0.0,39.000000,498.000000,NaN,NaN,...,681.000000,30.622204,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.135422 32.678614, -117.125674 3...",5
3133,2010,06073022000,189.0,0.000000,764.0,0.0,16.000000,1209.000000,NaN,NaN,...,1248.000000,7.371112,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.093518 32.68721, -117.090508 32...",1
3134,2010,06073022100,156.0,19.000000,788.0,0.0,1234.000000,2714.000000,NaN,NaN,...,3738.000000,67.584301,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-117.33373 33.144331, -117.31618 33....",5


In [4]:
cluster = SanDiego.cluster(
    columns=['p_nonhisp_white_persons'],
    n_clusters = 10,
    method ='kmeans',
)
cluster.gdf

KeyError: 1970